In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import json

def read_json_file(file_path):

  with open(file_path, 'r') as f:
    data = json.load(f)
  return data


# Replace these with your actual file paths
file_path1 = "/content/drive/My Drive/LLM_RESUME_DATA/Zero_shot/google-gemma-2-9b-it/gemma 2 9b it results.json"
file_path2 = '/content/drive/My Drive/LLM_RESUME_DATA/Zero_shot/lama_3_8b_100/result.json'
file_path3 = '/content/drive/My Drive/LLM_RESUME_DATA/Zero_shot/gemini_pro_100/result.json'

# Read the JSON files
data1 = read_json_file(file_path1)
data2 = read_json_file(file_path2)
data3 = read_json_file(file_path3)


In [ ]:
import pandas as pd

# Helper function to flatten dictionaries and handle lists by replacing them with their average
def flatten_dict_with_avg(d, parent_key='', sep='_'):
    items = []
    for k, v in d.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            # Recursively flatten if the value is a dictionary
            items.extend(flatten_dict_with_avg(v, new_key, sep=sep).items())
        elif isinstance(v, list):
            # Replace list with its average and add "_avg" to the key name
            avg_value = sum(v) / len(v) if len(v) > 0 else 0
            items.append((f"{new_key}_avg", avg_value))
        else:
            # Normal key-value pair
            items.append((new_key, v))
    return dict(items)

# Flatten the dictionaries and replace lists with averages
flattened_data1 = flatten_dict_with_avg(data1)
flattened_data2 = flatten_dict_with_avg(data2)
flattened_data3 = flatten_dict_with_avg(data3)

# Create a DataFrame from the flattened dictionaries
rows = []

# Iterate over the keys of the flattened dictionaries
for key in flattened_data1.keys():
    row = {'Key': key, 'Gemma-2-9b-it': flattened_data1[key], 'LLama-3-8b': flattened_data2[key], 'Gemini-Pro': flattened_data3[key]}
    rows.append(row)

df = pd.DataFrame(rows)

# Set 'Key' as the index before transposing
df.set_index('Key', inplace=True)

# Transpose the DataFrame
df_transposed = df.T

# Extract the column names and separate into 'Before' and 'After' groups
columns_before = [col for col in df_transposed.columns if 'Before' in col]
columns_after = [col for col in df_transposed.columns if 'After' in col]

# Sort columns by common key (without "Before" or "After") and interleave them
ordered_columns = []
for i in range(len(columns_before)):
    ordered_columns.append(columns_before[i])
    ordered_columns.append(columns_after[i])

# Reorder the DataFrame with the interleaved columns
df_transposed = df_transposed[ordered_columns]

# Reset index if needed (optional)
df_transposed.reset_index(inplace=True)

df_transposed


Key,index,Average Skill Match Proportion Before,Average Skill Match Proportion After,Median Skill Match Proportion Before,Median Skill Match Proportion After,Semantics Before_precision_avg,Semantics After_precision_avg,Semantics Before_recall_avg,Semantics After_recall_avg,Semantics Before_f1_avg,...,BLEU Before_precisions_avg,BLEU After_precisions_avg,BLEU Before_brevity_penalty,BLEU After_brevity_penalty,BLEU Before_length_ratio,BLEU After_length_ratio,BLEU Before_translation_length,BLEU After_translation_length,BLEU Before_reference_length,BLEU After_reference_length
0,Gemma-2-9b-it,0.046847,0.08306,0.0,0.0,0.804793,0.795942,0.813162,0.794264,0.808916,...,0.094207,0.094734,1.0,0.994725,1.097339,0.994739,285747,259030,260400,260400
1,LLama-3-8b,0.067264,0.35593,0.023256,0.258065,0.804793,0.820352,0.813162,0.836669,0.808916,...,0.094207,0.151654,1.0,1.0,1.097339,1.253372,285747,326378,260400,260400
2,Gemini-Pro,0.057937,0.393486,0.035714,0.303571,0.834119,0.8375,0.83716,0.842639,0.835619,...,0.333241,0.426311,1.0,1.0,3.588911,3.049494,96513,82007,26892,26892


In [ ]:
# save in a csv in current path of drive
df_transposed.to_csv('/content/drive/My Drive/LLM_RESUME_DATA/Zero_shot/combined_results.csv', index=False)
